In [ ]:
import pandas as pd

teste_df = pd.read_excel("teste.xlsx")
cor_df = pd.read_excel("cor.xlsx")
categorias_df = pd.read_excel("categorias.xlsx")
marcas_df = pd.read_excel("marcas.xlsx")
fornecedores_df = pd.read_excel("fornecedores.xlsx")

teste_df = teste_df.rename(columns={'REF KINGS' : 'Referências', 'DESCRIÇÃO' : 'Nome do Produto', 'ATACADO' : 'Valor atacado', 'VAR' : 'Valor varejo'})
teste_df [['Categoria', 'Descrição']] = teste_df['Nome do Produto'].str.split(' ', n=1, expand=True)
teste_df['Referências'] = teste_df['Referências'].str.slice(0, -4)

teste_df ['Status do Produto'] = '1'
teste_df ['SKU'] = teste_df ['Referências']
teste_df ['Exibir produto na home'] = '0'
teste_df ['Peso do Produto'] = ''
teste_df ['Tamanho do Produto'] = ''
teste_df ['Área de Disponibilidade'] = '{P}'

teste_df = teste_df.merge(cor_df, on= 'COR').rename(columns={'ID' : 'Cor'})
teste_df = teste_df.merge(fornecedores_df, on= 'FORNECEDOR').rename(columns={'ID' : 'Fornecedor ID'})
teste_df = teste_df.merge(categorias_df, on= 'Categoria').rename(columns={'ID' : 'Categoria ID'})

# teste_df = teste_df [['referências' , 'SKU']]

print(teste_df.to_markdown())


|     | MATERIAL   | Referências   | FORNECEDOR   | ENTREGA   | Nome do Produto                       | COR       | TAMANHO   | FIT       |   Valor atacado |     MKP |   Valor varejo |   QTD TOTAL |   TOTAL | Categoria   | Descrição                    |   Status do Produto | SKU         |   Exibir produto na home | Peso do Produto   | Tamanho do Produto   |   Cor |   Fornecedor ID |   Categoria ID |
|----:|:-----------|:--------------|:-------------|:----------|:--------------------------------------|:----------|:----------|:----------|----------------:|--------:|---------------:|------------:|--------:|:------------|:-----------------------------|--------------------:|:------------|-------------------------:|:------------------|:---------------------|------:|----------------:|---------------:|
|   0 | ALGODÃO    | CMT1INVER25   | YUKNALA      | MARÇO     | CAMISETA OVERSIZED FUTEBOL            | ROSA      | P         | OVERSIZED |         83.96   | 2.5     |          209.9 |         n